Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Check the GPU device you are using

In [ ]:
!nvidia-smi

Wed Aug  2 04:45:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%cd /content/gdrive/MyDrive

/content/gdrive/MyDrive


Git Clone the Alpaca-Lora Project

In [ ]:
!git clone https://github.com/tloen/alpaca-lora.git

Cloning into 'alpaca-lora'...
remote: Enumerating objects: 607, done.
remote: Total 607 (delta 0), reused 0 (delta 0), pack-reused 607
Receiving objects: 100% (607/607), 27.84 MiB | 15.00 MiB/s, done.
Resolving deltas: 100% (357/357), done.


In [ ]:
%cd alpaca-lora

/content/alpaca-lora


Install the package

In [ ]:
!pip install -r requirements.txt

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-nlhdhpri
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-nlhdhpri
  Resolved https://github.com/huggingface/peft.git to commit b10a8cedf612e3994dc67fe8dad2a40c9076bcd4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 101.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Fine-tunning on the Llama-7b.
One tricky part is to tune the micro_batch_size such that your memory can be fully utilized. Here I choose it to be 50 for 40GB-A100. You may want to optimize it by yourself.

## Downsizing alpaca dataset to 15k and saving to disk

In [ ]:
from datasets import load_dataset

data = load_dataset("json", data_files="alpaca.json")
data = data.shuffle()
data
new_data_size = 1000 ## make alpaca data smaller
data['train'] = data['train'].select(range(new_data_size))

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 1000
    })
})

In [ ]:
data['train']['instruction']

In [ ]:
dataset1 = data['train']

In [ ]:
from datasets import load_dataset
dolly_data = load_dataset("databricks/databricks-dolly-15k")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dolly_data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 15011
    })
})

In [ ]:
import json

data_list = []
for example in dolly_data['train']:
    data_dict = {
        'instruction': example['instruction'],
        'input': example['context'], ## change context to input for alpaca dataset
        'output': example['response'] ## change context to response for alpaca dataset
    }
    data_list.append(data_dict)

# Write the list of dictionaries to a JSON file
output_file = 'databricks-dolly-dataset.json'
with open(output_file, 'w') as json_file:
    json.dump(data_list, json_file, indent=4)

In [ ]:
import json

# Convert dataset to a list of dictionaries
data_list = []
data_dict = {}
for feature in data['train'].features:
    data_dict[feature] = data['train'][feature]
data_list.append(data_dict)

# Write the list of dictionaries to a JSON file
output_file = 'alpaca-1k.json'
with open(output_file, 'w') as json_file:
    json.dump(data_list, json_file)

In [ ]:
# data.save_to_disk('alpaca-15k')

Saving the dataset (0/1 shards):   0%|          | 0/15000 [00:00<?, ? examples/s]

In [ ]:
data['train'].to_json("squad.json")

Creating json from Arrow format:   0%|          | 0/15 [00:00<?, ?ba/s]

6018561

In [ ]:
data['train']

## Train

In [ ]:
!pip install wandb
!wandb login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=cc5f7af3a0fd7a071dd3944562ebbd4884751ec0e00f22e07ed328cc08e9292f
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
## 32 micro_bs for 3b model occupied 9 gb of vram
!python finetune.py \
    --base_model 'abhishek/llama-2-7b-hf-small-shards' \
    --data_path '/content/regen-100-filtered.json' \
    --output_dir '/content/code_alpaca_lora' \
    --batch_size 32 \
    --micro_batch_size 8 \
    --num_epochs 2 \
    --learning_rate 1e-4 \
    --cutoff_len 512 \
    --val_set_size 20 \
    --lora_r 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --lora_target_modules '[q_proj,v_proj]' \
    --train_on_inputs \
    --group_by_length \
    --wandb_project llama-alpaca-test \
    --wandb_log_model "true" \
    --wandb_run_name "code-alpaca-100-self" \
    #  --resume_from_checkpoint True \

2023-08-02 05:00:53.012361: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Training Alpaca-LoRA model with params:
base_model: abhishek/llama-2-7b-hf-small-shards
data_path: /content/regen-100-filtered.json
output_dir: /content/code_alpaca_lora
batch_size: 32
micro_batch_size: 8
num_epochs: 2
learning_rate: 0.0001
cutoff_len: 512
val_set_size: 20
lora_r: 8
lora_alpha: 16
lora_dropout: 0.05
lora_target_modules: ['q_proj', 'v_proj']
train_on_inputs: True
add_eos_token: False
group_by_length: True
wandb_project: llama-alpaca-test
wandb_run_name: code-alpaca-100-self
wandb_watch: 
wandb_log_model: true
resume_from_checkpoint: False
prompt template: alpaca

Loading checkpoint shards: 100% 10/10 [02:22<00:00, 14.25s/it]
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:116: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from huggingface_hub import create_repo
create_repo("sandeshrajx/code-alpaca-100")

RepoUrl('https://huggingface.co/sandeshrajx/code-alpaca-100', endpoint='https://huggingface.co', repo_type='model', repo_id='sandeshrajx/code-alpaca-100')

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

# Upload all the content from the local folder to your remote Space.
# By default, files are uploaded at the root of the repo
api.upload_folder(
    folder_path="/content/code_alpaca_lora",
    repo_id="sandeshrajx/code-alpaca-100",
)

adapter_model.bin:   0%|          | 0.00/443 [00:00<?, ?B/s]

'https://huggingface.co/sandeshrajx/code-alpaca-100/tree/main/'

In [ ]:
type(resume_from_checkpoint)

str

In [ ]:
LlamaForCausalLM.from_pretrained??

In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer
import torch
import transformers
model = LlamaForCausalLM.from_pretrained(
        "abhishek/llama-2-7b-hf-small-shards",
        load_in_8bit=True,
        torch_dtype=torch.float16,
        device_map="auto",
    )


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

## Test Inference

In [ ]:
# from transformers import AutoModelForSeq2SeqLM
# from peft import PeftModel, PeftConfig

#   peft_model_id = "smangrul/twitter_complaints_bigscience_T0_3B_LORA_SEQ_2_SEQ_LM"
#   config = PeftConfig.from_pretrained(peft_model_id)
#   model = AutoModelForCasualLM.from_pretrained(config.base_model_name_or_path)
# + model = PeftModel.from_pretrained(model, peft_model_id)
#   tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

#   model = model.to(device)
#   model.eval()
#   inputs = tokenizer("Tweet text : @HondaCustSvc Your customer service has been horrible during the recall process. I will never purchase a Honda again. Label :", return_tensors="pt")

#   with torch.no_grad():
#       outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=10)
#       print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])
# # 'complaint'

In [ ]:
import torch

In [ ]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model
from transformers import GenerationConfig

# model = AutoModelForCausalLM.from_pretrained(
#   "openlm-research/open_llama_3b",
#     load_in_8bit=True,
#     device_map="auto",
# )
tokenizer = AutoTokenizer.from_pretrained(
    "abhishek/llama-2-7b-hf-small-shards"
)

In [ ]:
# tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
model = PeftModel.from_pretrained(model, "/content/code_alpaca_lora")

## Inference

In [ ]:
PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Generate a Python script to find the maximum of two numbers.
### Response:"""
# PROMPT = "What is the meaning of life?"
inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
)

In [ ]:
%%time
from transformers import GenerationConfig

input_ids = inputs["input_ids"].cuda()

generation_config = GenerationConfig(
    temperature=1,
    top_p=0.95,
    repetition_penalty=1.15,
    num_return_sequences=1
    # pad_token_id=tokenizer.eos_token_id
)

print("Generating... Without LORA model")

generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=512,
)
for s in generation_output.sequences:
    print(len(s))
    print(tokenizer.decode(s))
    print ("#####################")

Generating... Without LORA model
75
<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Generate a Python script to find the maximum of two numbers.
### Response:
```python
def max(a, b):
    if a > b: return a
    else: return b
```</s>
#####################
CPU times: user 7.71 s, sys: 13.9 ms, total: 7.73 s
Wall time: 8.05 s


In [ ]:
%%time
from transformers import GenerationConfig

input_ids = inputs["input_ids"].cuda()

generation_config = GenerationConfig(
    temperature=1,
    top_p=0.95,
    repetition_penalty=1.15,
    num_return_sequences=1
    # pad_token_id=tokenizer.eos_token_id
)
print("Generating... WITH LORA model")

generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=512,
)
for s in generation_output.sequences:
    print(len(s))
    print(tokenizer.decode(s))
    print ("#####################")

Generating... WITH LORA model
75
<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Generate a Python script to find the maximum of two numbers.
### Response:
```python
def max(a, b):
    if a > b: return a
    else: return b
```</s>
#####################
CPU times: user 7.83 s, sys: 6.52 ms, total: 7.84 s
Wall time: 8.08 s


In [ ]:
%%time
from transformers import GenerationConfig

input_ids = inputs["input_ids"].cuda()

generation_config = GenerationConfig(
    temperature=1,
    top_p=0.95,
    repetition_penalty=1.15,
    num_return_sequences=1
    # pad_token_id=tokenizer.eos_token_id
)
print("Generating...")

generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    max_new_tokens=512,
)
for s in generation_output.sequences:
    print(len(s))
    print(tokenizer.decode(s))

Generating...
520
<s> What is the meaning of life? This question has been asked by many people throughout history. It’s a question that can be difficult to answer, but it’s one that we should all ask ourselves from time to time. In this blog post, I will explore what the Bible says about the meaning of life and how you can find your own purpose in God.
The Meaning Of Life According To The Bible
What Does The Bible Say About Purpose And Meaning?
How Can You Find Your Own Purpose In God?
There are many different interpretations of what the Bible says about the meaning of life. Some believe that the ultimate goal is to live a good life and do good deeds for others. Others believe that the meaning of life is to have faith in Jesus Christ as our Savior and Lord. Still others believe that the meaning of life is simply to enjoy every moment and not worry too much about the future or past. Ultimately, each person must decide for themselves what they think the true meaning of life is. However, 